In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
from transformers import (
    AutoConfig, 
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TextStreamer, 
    GenerationConfig, 
    logging,
    TrainingArguments,
    Trainer,
)
import datasets
import json
import pandas as pd
from pathlib import Path
import torch
import transformers
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

2023-12-02 23:58:51.321375: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 23:58:51.321406: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 23:58:51.321433: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-02 23:58:51.327876: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Setup Dataset

In [4]:
_OPENFUNCTIONS_TEST = "datasets/gorilla_openfunctions/test.jsonl"
_OPENFUNCTIONS_TRAIN = "datasets/gorilla_openfunctions/train.jsonl"

Base Zephyr Model Prompt Template:
```text
<|system|>
You are a friendly chatbot who always responds in the style of a pirate.</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
Ah, me hearty matey! But yer question be a puzzler! A human cannot eat a helicopter in one sitting, as helicopters are not edible. They be made of metal, plastic, and other materials, not food!
```

In [9]:
test_data = pd.read_json(_OPENFUNCTIONS_TEST, lines=True)
train_data = pd.read_json(_OPENFUNCTIONS_TRAIN, lines=True)

In [ ]:
column_types = {
    'question': 'string',
    'function': 'string',
    'model_answer': 'string',
}
test_data = test_data.astype(column_types)
train_data = train_data.astype(column_types)

In [ ]:
train_data['Functions'][432]

## Train Model

In [3]:
logging.set_verbosity_info()

In [4]:
_BASE_MODEL_PATH = Path('../models/zephyr-7b-beta/')
_LORA_OUTPUT_PATH = Path('../models/loras/')

In [5]:
base_model_tokenizer = AutoTokenizer.from_pretrained(_BASE_MODEL_PATH, use_fast=False)
base_model_config = AutoConfig.from_pretrained(_BASE_MODEL_PATH)

loading file tokenizer.model
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file tokenizer.json
loading configuration file ../models/zephyr-7b-beta/config.json
Model config MistralConfig {
  "_name_or_path": "../models/zephyr-7b-beta",
  "architectures": [
    "MistralForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 2,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.34.1",
  "use_cache": true,
  "vocab_size": 32000
}



In [7]:
base_model = AutoModelForCausalLM.from_pretrained(
    _BASE_MODEL_PATH,
    config=base_model_config, 
    device_map='cuda', 
    torch_dtype=base_model_config.torch_dtype,
    low_cpu_mem_usage=True
)

loading weights file ../models/zephyr-7b-beta/pytorch_model.bin.index.json
Instantiating MistralForCausalLM model under default dtype torch.bfloat16.
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 2
}



Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing MistralForCausalLM.

All the weights of MistralForCausalLM were initialized from the model checkpoint at ../models/zephyr-7b-beta.
If your task is similar to the task the model of the checkpoint was trained on, you can already use MistralForCausalLM for predictions without further training.
loading configuration file ../models/zephyr-7b-beta/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}



In [8]:
for param in base_model.parameters():
    # Turning off gradient calculation for base model as we want to train lora, not base model
    param.requires_grad = False

In [9]:
base_model.config.use_cache = False

In [10]:
base_model_tokenizer.bos_token, base_model_tokenizer.pad_token, base_model_tokenizer.eos_token, base_model_tokenizer.unk_token

('<s>', '</s>', '</s>', '<unk>')

### [Gradient Accumulation](https://huggingface.co/docs/transformers/v4.18.0/en/performance#gradient-accumulation)
    The idea behind gradient accumulation is to instead of calculating the gradients for the whole batch at once to do it in smaller steps. The way we do that is to calculate the gradients iteratively in smaller batches by doing a forward and backward pass through the model and accumulating the gradients in the process. When enough gradients are accumulated we run the model’s optimization step. This way we can easily increase the overall batch size to numbers that would never fit into the GPU’s memory. In turn, however, the added forward and backward passes can slow down the training a bit.

### [Gradient Checkpointing](https://huggingface.co/docs/transformers/v4.18.0/en/performance#gradient-checkpointing)
    Even when we set the batch size to 1 and use gradient accumulation we can still run out of memory when working with large models. In order to compute the gradients during the backward pass all activations from the forward pass are normally saved. This can create a big memory overhead. Alternatively, one could forget all activations during the forward pass and recompute them on demand during the backward pass. This would however add a significant computational overhead and slow down training.

    Gradient checkpointing strikes a compromise between the two approaches and saves strategically selected activations throughout the computational graph so only a fraction of the activations need to be re-computed for the gradients. See this great article explaining the ideas behind gradient checkpointing.

In [11]:
base_model.gradient_checkpointing_enable()
base_model.enable_input_require_grads()

In [12]:
def print_trainable_parameters(model: nn.Module):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for name, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable(%): {100 * trainable_params / all_param}"
    )

https://medium.com/@manyi.yim/more-about-loraconfig-from-peft-581cf54643db

In [13]:
lora_config = LoraConfig(
    # peft_type: str | PeftType = None,
    # auto_mapping: dict | None = None,
    # base_model_name_or_path: str = None,
    # revision: str = None,
    task_type = TaskType.CAUSAL_LM,
    # inference_mode: bool = False,
    r = 64, #! 8, 16, 32, 64
    target_modules = ["q_proj", "v_proj"],
    lora_alpha = 16, #! 8, 16, 32
    lora_dropout = 0.1, #! 0.05
    # fan_in_fan_out: bool = False,
    bias = "none",
    # modules_to_save: List[str] | None = None,
    # init_lora_weights: bool = True,
    # layers_to_transform: List[int] | int | None = None,
    # layers_pattern: str | None = None
)
peft_model = get_peft_model(base_model, lora_config)
print_trainable_parameters(peft_model)

trainable params: 27262976 || all params: 7268995072 || trainable(%): 0.3750583915652433


In [14]:
trainig_parms = TrainingArguments(
    output_dir=_LORA_OUTPUT_PATH,
    num_train_epochs=1,
    gradient_accumulation_steps=1,
    per_device_train_batch_size=4,
    
    logging_steps=25, # Default: 500
    fp16=True,
    
    save_steps=25,
    save_safetensors=True,
    report_to="tensorboard",
)

PyTorch: setting up devices


In [15]:
# Sample dataset
data = datasets.load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: base_model_tokenizer(samples['quote']), batched=True)

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [16]:
trainer = Trainer(
    model=peft_model,
    train_dataset=data['train'],
    # eval_dataset=data['validation'],
    args=trainig_parms,
    tokenizer=base_model_tokenizer,
    # callbacks=[],
    data_collator=transformers.DataCollatorForLanguageModeling(base_model_tokenizer, mlm=False),
)

You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.


In [17]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: author, tags, quote. If author, tags, quote are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 2,508
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 627
  Number of trainable parameters = 27,262,976
/home/devasheesh/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the 

Step,Training Loss
25,1.541300
50,1.603000
75,1.573500
100,1.266300


Saving model checkpoint to ../models/loras/checkpoint-25
tokenizer config file saved in ../models/loras/checkpoint-25/tokenizer_config.json
Special tokens file saved in ../models/loras/checkpoint-25/special_tokens_map.json
/home/devasheesh/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Saving model checkpoint to ../models/loras/checkpoint-50
tokenizer config file saved in ../models/loras/checkpoint-50/tokenizer_config.json
Special tokens file saved in ../models/loras/checkpoint-50/special_tokens_map.json
/home/devasheesh/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: Us

KeyboardInterrupt: 